In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import 用到的套件

In [2]:
import numpy as np
import pandas as pd
import time
import torch
import torch.nn.functional as F
from torchvision import transforms,models
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")
import sys
sys.path.append(folder+"模組/")
import webcam

## 是否有GPU可以使用

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 測試資料預處理


In [4]:
test_transform=transforms.Compose([
    transforms.Resize(256),#改變大小
    transforms.CenterCrop(224),#中央裁剪
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])#像素資料歸一化
])

## 載入自己訓練的模型

In [5]:
model = torch.load(folder+"資料集/水果30分類/fruit30.pth")
model=model.to(device)          #是否使用GPU
model.eval()                #設定為測試模式
model                   #查看模型

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 載入水果30分類的索引對應

In [6]:
index2label= np.load(folder+"資料集/水果30分類/index2label.npy", allow_pickle=True).item()

In [7]:
index2label


{0: '哈密瓜',
 1: '奇異果',
 2: '小黃瓜',
 3: '山竹',
 4: '柚子',
 5: '柳橙',
 6: '梨',
 7: '椰子',
 8: '楊梅',
 9: '榴槤',
 10: '檸檬',
 11: '櫻桃',
 12: '火龍果',
 13: '白葡萄',
 14: '石榴',
 15: '砂糖橘',
 16: '紅番茄',
 17: '紅葡萄',
 18: '紅蘋果',
 19: '聖女番茄',
 20: '胡蘿蔔',
 21: '芒果',
 22: '苦瓜',
 23: '草莓',
 24: '荔枝',
 25: '西瓜',
 26: '青蘋果',
 27: '香蕉',
 28: '鳳梨',
 29: '龍眼'}

## 即時物品影像辨識

In [ ]:
webcam.video_stream()   #開始從webcam串流影像
count = 0
Starttime=time.time()
HTML=""         #影片上方要顯示的文字 
while True:
  img = webcam.video_frame(HTML)          #取得目前的影像frame，傳入對應的圖片說明
  if not img:
    break
  img = test_transform(img).unsqueeze(0).to(device) #目前的影像作預處裡
  output = model(img)                 #做前向傳播，得到各分類的分數
  softmax_out = F.softmax(output,dim=1).squeeze()   #對各分類做softmax運算，得到分類機率
  values,indexs = torch.topk(softmax_out,5)     #取可信度最大的5個结果：分類分數，分類索引
  Output=""
  for i in range(len(indexs)):
    Output +="{}({:.1f}%)".format(index2label[int(indexs[i])],values[i].item()*100)
  count+=1
  Stoptime=time.time()
  HTML =str(round(count/(Stoptime-Starttime),2))+"FPS<br>"+Output

<IPython.core.display.Javascript object>